In [ ]:
import torch
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# import h5py
import scipy.io as scio
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [ ]:
X = []  # 定义图像名称
Y = []  # 定义图像分类类标

with open('fc_list.txt','r') as f:
    data=f.readlines()
data=[item.replace('\n','').split('\t') for item in data]
random.seed(2022)
random.shuffle(data)
result_index_matrix=torch.zeros(5,3,3)
result=[]

k_fold=5
for kk in range(k_fold):
    k_fold_len = int(len(data)//k_fold)
    test_data = data[k_fold_len*kk:k_fold_len*(kk+1)]
    train_data = data[:k_fold_len*kk] + data[k_fold_len*(kk+1):]

    X_train=[x[0] for x in train_data]
    y_train=[x[1] for x in train_data]
    X_test=[x[0] for x in test_data]
    y_test=[x[1] for x in test_data]
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    X_test=np.array(X_test)
    y_test=np.array(y_test)

    XX_train = []
    for imagepath in X_train:
        # temp_image=h5py.File(imagepath,'r')
        temp_image=scio.loadmat(imagepath)['fc']
        XX_train.append(temp_image.flatten())  #reshape into 1D array

    # 测试集
    XX_test = []
    for imagepath in X_test:
        # temp_image=h5py.File(imagepath,'r')
        temp_image=scio.loadmat(imagepath)['fc']
        XX_test.append(temp_image.flatten())
        

    # clf = DecisionTreeClassifier(random_state=0) #single tree
    rfc = RandomForestClassifier(random_state=0) #random tree
    # clf = clf.fit(XX_train,y_train)
    rfc = rfc.fit(XX_train,y_train)
    prediction = rfc.predict(XX_test)  

    # score_c = clf.score(XX_test,y_test)
    score_r = rfc.score(XX_test,y_test)
    # print("Single Tree:",score_c)
    print("Random Forest:",score_r)

    c_matrix=confusion_matrix(y_test,prediction)
    for idm in range(3):
        result_index_matrix[kk][0][idm]=c_matrix[idm][idm]/np.sum(c_matrix[:,idm])
        result_index_matrix[kk][1][idm]=c_matrix[idm][idm]/np.sum(c_matrix[idm,:])
        result_index_matrix[kk][2][idm]=(np.sum(c_matrix)-np.sum(c_matrix[:,idm])-np.sum(c_matrix[idm,:])+c_matrix[idm][idm])/(np.sum(c_matrix)-np.sum(c_matrix[idm,:])) 
    print(classification_report(y_test, prediction))   
    acc = accuracy_score(y_test,prediction)  
    print("acc: ", acc)
    print(result_index_matrix[kk])

    result.append(acc)
print('mean:{:.4f}  std:{:.4f}'.format(np.mean(result),np.std(result)))
mean_matrix=result_index_matrix.sum(dim=0)/k_fold
print(mean_matrix)
print(mean_matrix.sum(dim=1)/3)
